.. _AWS Connect:

AWS Connect
--------------


Initial steps to set up a Jupyter notebook using AWS. 


## Create VPC for your notebooks

1) Navigate to VPC in dashboard: 
*   You can search for VPC or find it under Networking & Content Delivery

2) This brings you to VPC Dashboard: 

![VPC Dashboard](../_static/images/vpc_dashboard.png)

3) Click `Your VPCs` and then `Create VPC`. 

4) In `Create VPC` you will need to fill in: 

![Create VPC](../_static/images/create_vpc.png)

Name: awsFromScratch

IPv4: 10.0.0.0/24

IPv6: None

Tenancy: Default


## Create Internet Gateway

1) Navigate in left menu to `Internet Gateways`

2) Create Internet Gateway

3) Enter:
*   Name: <IG_Name>

4)  Navigate to actions:
*   Attach to VPC: <VPC_Name>

## Create Routing Table

1) Navigate in left menu to `Route Tables`

2) Create route table

3) Enter:
*   Name: <RT_Name>
*   VPC: Attach <VPC_Name>

4) Navigate to actions and select edit routes:
*  Destination: 0.0.0.0/0
*  Target: <IG_Name>

## Set up Security Groups

1) Navigate in left menu to `Security` -> `Security Groups`:

![Nav to Security Group](../_static/images/create_SG.png)

2) Fill in Security group name, description and connect it to your VPC you created before: 

![Create Security Group](../_static/images/create_SG_2.png)

## Update rules

1) In `Security Groups` dashboard, select your newly minted security group and click `Actions` then select `Edit InboundRules`.

2) Select Add Rule and input:
![Create Security Group](../_static/images/edit_inbound_rules.png)

*   Note: ssh is so you can log in to instance via terminal. 
          custom TCP is so jupyter notebook can connect via browser. 

3) Click `Save Rules`.


## Create subnet

1) 


## Allocate an elastic IP

1) 

